In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split

from pgmpy.estimators import HillClimbSearch, ExhaustiveSearch, BayesianEstimator
from pgmpy.estimators import ConstraintBasedEstimator, K2Score, BicScore, BDeuScore
from pgmpy.estimators import MaximumLikelihoodEstimator

from pgmpy.models import BayesianModel

np.random.seed(359)

In [20]:
def LL(x,model,verbose=False):
    loglike = 0
    for cpd in model.get_cpds():
        temp_cpd = cpd.copy()
        thevariable = temp_cpd.variable
        theparents = model.predecessors(thevariable)
        for parent in theparents:
            temp_cpd.reduce([(parent, x[parent])])
        if x[thevariable] < len(temp_cpd.get_values()): # I added this to stop it from failing
#             print("HERE", x, thevariable, temp_cpd.get_values())
            try:
                theprob = temp_cpd.get_values()[x[thevariable],0]
                if verbose:
                    print (thevariable,theparents,theprob)
                loglike += np.log(theprob)
            except:
                pass #print('Error',  x, thevariable)
    return loglike

In [21]:
def get_anomaly_ranks(data):

    for i in data.iloc[:,:]:
        data[i] = pd.cut(data[i], bins=10, labels=False)

    hc = HillClimbSearch(data, scoring_method = BicScore(data))
    bic_best_model = hc.estimate()
    best_edges = bic_best_model.edges()
    print("Edges: ")
    for edge in best_edges: 
        print(edge)

    model = BayesianModel( bic_best_model.edges() )
    model.fit(data, estimator=MaximumLikelihoodEstimator)
    exmp = data.apply(lambda x: LL(x, model), axis=1)
    exmp2=pd.Series(exmp)
    exmp2.index = data.index
    return exmp2

In [22]:
age_sex_data = pd.read_csv('../data/Demographics/Age_Sex/tract_age_sex_acs2018.csv')

age_sex_data['00_19'] = age_sex_data['00-05']+age_sex_data['05-09']+age_sex_data['10-14']+age_sex_data['15-19']
age_sex_data['20_34'] = age_sex_data['20-24']+age_sex_data['25-29']+age_sex_data['30-34']
age_sex_data['35_49'] = age_sex_data['35-39']+age_sex_data['40-44']+age_sex_data['45-49']
age_sex_data['50_64'] = age_sex_data['50-54']+age_sex_data['55-59']+age_sex_data['60-64']
age_sex_data['65_UP'] = age_sex_data['65-69']+age_sex_data['70-74']+age_sex_data['75-79']+age_sex_data['80-84']+age_sex_data['85-UP']

age_sex_data['Male'] = age_sex_data['Male'] / age_sex_data['Total']
age_sex_data['Female'] = age_sex_data['Female'] / age_sex_data['Total']
age_sex_data['00_19'] = age_sex_data['00_19'] / age_sex_data['Total']
age_sex_data['20_34'] = age_sex_data['20_34'] / age_sex_data['Total']
age_sex_data['35_49'] = age_sex_data['35_49'] / age_sex_data['Total']
age_sex_data['50_64'] = age_sex_data['50_64'] / age_sex_data['Total']
age_sex_data['65_UP'] = age_sex_data['65_UP'] / age_sex_data['Total']

age_sex_data = age_sex_data[['city', 'tract', 'county', 'BoroCTLbl', 'Total', # 'Male', 'Female',
                            '00_19', '20_34', '35_49', '50_64', '65_UP']]
age_sex_data = age_sex_data.replace(np.nan, 0.0)

age_sex_data.head()

,city,tract,county,BoroCTLbl,Total,00_19,20_34,35_49,50_64,65_UP
0,New York,1.0,Bronx,Bronx 1,7080,0.076412,0.449859,0.318503,0.148023,0.007203
1,New York,2.0,Bronx,Bronx 2,4542,0.247908,0.161823,0.199031,0.182078,0.209159
2,New York,4.0,Bronx,Bronx 4,5634,0.216365,0.225950,0.215122,0.216542,0.126021
3,New York,16.0,Bronx,Bronx 16,5917,0.275477,0.206693,0.168498,0.182187,0.167146
4,New York,19.0,Bronx,Bronx 19,2765,0.292586,0.318987,0.209403,0.151537,0.027486


In [23]:
income_data = pd.read_csv('../data/Demographics/income/household_income_acs2018.csv')

income_data['00_50k'] = income_data.loc[:,'HH 0-10k':'HH 35k-50k'].sum(axis=1) / 100
income_data['50_100k'] = income_data.loc[:,'HH 50k-75k':'HH 75k-100k'].sum(axis=1) / 100
income_data['100_150k'] = income_data['HH 100k-150k'] / 100
income_data['150_UP'] = income_data.loc[:,'HH 150k-200k':'HH 200k-UP'].sum(axis=1) / 100

income_data.head()

,city,tract,county,BoroCTLbl,Households (HH) Count,HH 0-10k,HH 10k-15k,HH 15k-25k,HH 25k-35k,HH 35k-50k,...,HH 75k-100k,HH 100k-150k,HH 150k-200k,HH 200k-UP,Household Median Income,Household Mean Income,00_50k,50_100k,100_150k,150_UP
0,New York,1.0,Bronx,Bronx 1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.000,0.000,0.000,0.000
1,New York,2.0,Bronx,Bronx 2,1328,6.9,2.4,12.3,9.9,11.5,...,9.5,15.4,5.9,5.9,59914,72979,0.430,0.299,0.154,0.118
2,New York,4.0,Bronx,Bronx 4,1963,7.2,1.9,7.6,3.5,8.5,...,17.2,17.3,12.6,6.3,82073,94723,0.287,0.352,0.173,0.189
3,New York,16.0,Bronx,Bronx 16,1982,8.4,12.5,11.8,16.5,15.1,...,8.1,7.7,1.2,1.1,35802,59663,0.643,0.259,0.077,0.023
4,New York,19.0,Bronx,Bronx 19,929,13.7,10.3,6.7,7.2,18.2,...,12.4,10.7,0.0,1.6,42075,54415,0.561,0.317,0.107,0.016


In [24]:
race_data = pd.read_csv('../data/Demographics/Basic Count/tract_race_acs2018.csv')
race_data['White'] = race_data['White'] / race_data['Total']
race_data['Black'] = race_data['Black'] / race_data['Total']
race_data['Native'] = race_data['Native'] / race_data['Total']
race_data['Asian'] = race_data['Asian'] / race_data['Total']
race_data['Pacific Islander'] = race_data['Pacific Islander'] / race_data['Total']
race_data['Other'] = race_data['Other'] / race_data['Total']
race_data['Two or More'] = race_data['Two or More'] / race_data['Total']
race_data = race_data.replace(np.nan, 0.0)
race_data.head()

,city,tract,county,BoroCTLbl,Total,White,Black,Native,Asian,Pacific Islander,Other,Two or More
0,New York,429.02,Bronx,Bronx 429.02,4205,0.115339,0.329845,0.042568,0.047087,0.0,0.437337,0.027824
1,New York,330.00,Bronx,Bronx 330,5885,0.499745,0.152421,0.000000,0.000000,0.0,0.347833,0.000000
2,New York,358.00,Bronx,Bronx 358,8054,0.096722,0.739757,0.000000,0.022349,0.0,0.109635,0.031537
3,New York,371.00,Bronx,Bronx 371,4322,0.259602,0.341046,0.003008,0.004165,0.0,0.338501,0.053679
4,New York,385.00,Bronx,Bronx 385,4757,0.072314,0.319739,0.007358,0.000000,0.0,0.561068,0.039521


In [25]:
internet_data = pd.read_csv("../data/ACS_Internet_Subscription/subscription_acs_2018.csv")
internet_data['Mobile_Dependent'] = internet_data['Mobile_Dependent'] / internet_data['Total']
internet_data['Wired_Broadband'] = internet_data['Wired_Broadband'] / internet_data['Total']
internet_data['No_Internet'] = internet_data['No_Internet'] / internet_data['Total']
internet_data = internet_data.replace(np.nan, 0.0)
internet_data.head()

,city,tract,county,BoroCTLbl,Total,Mobile_Dependent,Wired_Broadband,No_Internet
0,New York,429.02,Bronx,Bronx 429.02,1599,0.090056,0.626642,0.262039
1,New York,330.00,Bronx,Bronx 330,2129,0.042273,0.693753,0.191170
2,New York,358.00,Bronx,Bronx 358,2443,0.081867,0.714286,0.100287
3,New York,371.00,Bronx,Bronx 371,1739,0.054054,0.533065,0.361127
4,New York,385.00,Bronx,Bronx 385,1674,0.093787,0.525687,0.280765


In [26]:
def gen_borocode(row):
    if row['tract']%100 == 0:
        return row['boro'].capitalize() + " " + str(row['tract']//100)  
    else:
        return row['boro'].capitalize() + " " + str(row['tract']//100) + "." + str((row['tract']%100)).zfill(2)    

fcc_data = pd.read_csv("../data/FCC_477/NY-Fixed-Jun2018-v1.csv")

fcc_data = fcc_data[(fcc_data['Max Advertised Downstream Speed (mbps)'] >= 25) 
         & (fcc_data['Technology Code'].isin(['Fiber', 'Cable']))]


fcc_data['BoroCTLbl'] = fcc_data.apply(gen_borocode, axis = 1) 

fcc_providers = fcc_data[['FRN','BoroCTLbl']].groupby(by='BoroCTLbl', as_index=False).agg({'FRN': pd.Series.nunique})
fcc_providers = fcc_providers.rename(columns={'FRN':'Unique_ISPs'})
fcc_providers.head()

,BoroCTLbl,Unique_ISPs
0,Bronx 1,1
1,Bronx 110,2
2,Bronx 115.02,2
3,Bronx 117,2
4,Bronx 118,2


In [48]:
data = pd.merge(internet_data, 
                income_data[['BoroCTLbl','00_50k','50_100k','100_150k','150_UP']], 
                how='left', 
                on=['BoroCTLbl'])[['BoroCTLbl', 'Mobile_Dependent','Wired_Broadband','No_Internet',
                                    '00_50k', '50_100k', '100_150k','150_UP']] #.drop(columns=['Households (HH) Count'])

data = pd.merge(data, 
                race_data[['BoroCTLbl','White','Black','Native','Asian','Pacific Islander','Other','Two or More']], 
                how='left', 
                on=['BoroCTLbl'])

data = pd.merge(data, 
                fcc_providers,
                how='left', 
                on=['BoroCTLbl'])

data = pd.merge(data, 
                age_sex_data[['BoroCTLbl','00_19', '20_34', '35_49', '50_64', '65_UP']], 
                how='left', 
                on=['BoroCTLbl'])

data["Unique_ISPs"] = data["Unique_ISPs"].fillna(0)

data.head()

,BoroCTLbl,Mobile_Dependent,Wired_Broadband,No_Internet,00_50k,50_100k,100_150k,150_UP,White,Black,...,Asian,Pacific Islander,Other,Two or More,Unique_ISPs,00_19,20_34,35_49,50_64,65_UP
0,Bronx 429.02,0.090056,0.626642,0.262039,0.677,0.219,0.081,0.023,0.115339,0.329845,...,0.047087,0.0,0.437337,0.027824,2.0,0.252081,0.251367,0.226159,0.160999,0.109394
1,Bronx 330,0.042273,0.693753,0.191170,0.674,0.231,0.065,0.029,0.499745,0.152421,...,0.000000,0.0,0.347833,0.000000,2.0,0.229227,0.214613,0.251317,0.207816,0.097026
2,Bronx 358,0.081867,0.714286,0.100287,0.316,0.325,0.179,0.180,0.096722,0.739757,...,0.022349,0.0,0.109635,0.031537,2.0,0.210206,0.268314,0.148373,0.237398,0.135709
3,Bronx 371,0.054054,0.533065,0.361127,0.660,0.241,0.057,0.043,0.259602,0.341046,...,0.004165,0.0,0.338501,0.053679,3.0,0.245488,0.218880,0.179778,0.184868,0.170986
4,Bronx 385,0.093787,0.525687,0.280765,0.851,0.107,0.033,0.008,0.072314,0.319739,...,0.000000,0.0,0.561068,0.039521,2.0,0.354635,0.252680,0.167543,0.127181,0.097961


In [49]:
data.corr()

,Mobile_Dependent,Wired_Broadband,No_Internet,00_50k,50_100k,100_150k,150_UP,White,Black,Native,Asian,Pacific Islander,Other,Two or More,Unique_ISPs,00_19,20_34,35_49,50_64,65_UP
Mobile_Dependent,1.000000,-0.205614,0.160272,0.250205,0.170289,-0.032095,-0.185596,-0.294930,0.261400,0.068734,-0.011439,0.006245,0.257327,0.027840,0.027247,0.224771,0.072519,0.060158,0.096594,-0.064255
Wired_Broadband,-0.205614,1.000000,-0.459799,-0.281935,0.407483,0.560720,0.581876,0.402701,-0.140124,-0.026651,0.204878,-0.009606,-0.196897,0.111448,0.133709,-0.039168,0.285796,0.502846,0.377686,0.189756
No_Internet,0.160272,-0.459799,1.000000,0.746643,-0.054130,-0.382200,-0.506771,-0.198523,0.218406,0.060346,-0.104595,0.025867,0.296669,-0.039593,-0.093311,0.450946,-0.047226,-0.133903,-0.012507,0.102776
00_50k,0.250205,-0.281935,0.746643,1.000000,-0.103426,-0.523271,-0.644598,-0.355708,0.242765,0.108871,-0.052102,0.046100,0.545155,0.082258,-0.040878,0.499043,0.080158,-0.038307,0.009639,-0.027366
50_100k,0.170289,0.407483,-0.054130,-0.103426,1.000000,0.229199,-0.125045,-0.000694,0.102295,0.009212,0.141505,-0.041961,0.026564,0.037716,-0.004873,0.121615,0.092225,0.256014,0.371159,0.074748
100_150k,-0.032095,0.560720,-0.382200,-0.523271,0.229199,1.000000,0.398131,0.307085,-0.062669,-0.025116,0.121955,-0.020746,-0.342606,0.013746,0.008575,-0.151325,0.124134,0.316476,0.249725,0.141353
150_UP,-0.185596,0.581876,-0.506771,-0.644598,-0.125045,0.398131,1.000000,0.554056,-0.268622,-0.089432,0.056734,-0.013770,-0.451910,0.036165,0.163502,-0.279992,0.165718,0.267572,0.083469,0.190464
White,-0.294930,0.402701,-0.198523,-0.355708,-0.000694,0.307085,0.554056,1.000000,-0.680420,-0.088023,-0.017887,-0.024790,-0.393430,-0.045626,0.031766,-0.104555,0.072165,0.151106,0.028765,0.286753
Black,0.261400,-0.140124,0.218406,0.242765,0.102295,-0.062669,-0.268622,-0.680420,1.000000,0.006985,-0.429081,0.016289,-0.019583,-0.072335,-0.113188,0.190827,0.012439,-0.034342,0.128854,-0.077956
Native,0.068734,-0.026651,0.060346,0.108871,0.009212,-0.025116,-0.089432,-0.088023,0.006985,1.000000,-0.009176,0.047857,0.122971,0.053252,0.011644,0.084007,0.040656,0.014135,-0.025487,-0.038048


In [50]:
# anomaly_ranks = get_anomaly_ranks(data.iloc[:,1:])

In [68]:
tmp = data.iloc[:,1:].copy() 
for i in tmp:
    tmp[i] = pd.cut(tmp[i], bins=10, labels=False)
tmp.head()

,Mobile_Dependent,Wired_Broadband,No_Internet,00_50k,50_100k,100_150k,150_UP,White,Black,Native,Asian,Pacific Islander,Other,Two or More,Unique_ISPs,00_19,20_34,35_49,50_64,65_UP
0,2,6,2,6,2,0,0,1,3,1,0,0,5,0,4,3,3,3,2,1
1,0,6,1,6,2,0,0,4,1,0,0,0,4,0,4,3,2,4,2,0
2,1,7,1,3,3,1,2,0,7,0,0,0,1,0,4,3,3,2,3,1
3,1,5,3,6,2,0,0,2,3,0,0,0,4,1,7,3,2,3,2,1
4,2,5,2,8,1,0,0,0,3,0,0,0,7,0,4,5,3,2,1,0


In [69]:
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator,ConstraintBasedEstimator

In [70]:
hc = HillClimbSearch(tmp, scoring_method=BicScore(tmp))
best_model = hc.estimate()
print("# Edges: ", len(best_model.edges()))
for edge in best_model.edges():
    print("Edge:", edge)

# Edges:  16
Edge: ('Wired_Broadband', 'No_Internet')
Edge: ('Wired_Broadband', 'Mobile_Dependent')
Edge: ('Wired_Broadband', '35_49')
Edge: ('00_50k', 'Wired_Broadband')
Edge: ('00_50k', '100_150k')
Edge: ('00_50k', '50_100k')
Edge: ('00_50k', '00_19')
Edge: ('150_UP', '00_50k')
Edge: ('White', 'Asian')
Edge: ('White', '150_UP')
Edge: ('White', 'Unique_ISPs')
Edge: ('Black', 'White')
Edge: ('Black', 'Other')
Edge: ('00_19', '20_34')
Edge: ('20_34', '50_64')
Edge: ('20_34', '65_UP')


In [71]:
model = BayesianModel(best_model.edges())
model.fit(tmp, estimator=MaximumLikelihoodEstimator)
exmp = tmp.apply(lambda x: LL(x, model), axis=1)
exmp2=pd.Series(exmp)
exmp2.index=data['BoroCTLbl']

C:\Users\erikl\Anaconda2\envs\AppliedDataScience\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


In [72]:
exmp2.sort_values().head(-5) #doesn't work due to zero/divide error

BoroCTLbl
Brooklyn 676         -inf
Queens 446.01        -inf
Queens 1205          -inf
Brooklyn 267         -inf
Brooklyn 529         -inf
                   ...   
Queens 107.01   -9.581451
Queens 613.02   -9.581451
Manhattan 311   -9.581451
Queens 37       -9.581451
Brooklyn 175    -9.581451
Length: 2162, dtype: float64

In [73]:
data[data['BoroCTLbl'] == 'Queens 1205']

,BoroCTLbl,Mobile_Dependent,Wired_Broadband,No_Internet,00_50k,50_100k,100_150k,150_UP,White,Black,...,Asian,Pacific Islander,Other,Two or More,Unique_ISPs,00_19,20_34,35_49,50_64,65_UP
1737,Queens 1205,0.089756,0.418537,0.441951,0.778,0.124,0.078,0.02,0.076055,0.045852,...,0.850437,0.0,0.0,0.019651,2.0,0.152475,0.141557,0.178311,0.232169,0.295488
